In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

In [109]:
# Make dataframe from files with articles
df = pd.DataFrame()
onlyfiles = [f for f in listdir(r'files/') if isfile(join(r'files/', f))]
row = -1
for i in onlyfiles:
    file = pd.read_pickle(r'files/'+i)
    row = row + 1
    for key in file.keys():
        df.loc[row,key] = file[key]

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122720 entries, 0 to 122719
Data columns (total 11 columns):
id          122720 non-null float64
status      122720 non-null object
lang        59305 non-null object
title       59305 non-null object
text        59305 non-null object
theme       59305 non-null object
keywords    59305 non-null object
time        59305 non-null object
comcount    59305 non-null object
autor       59305 non-null object
full_url    59305 non-null object
dtypes: float64(1), object(10)
memory usage: 16.2+ MB


In [116]:
id_min = df.id.min()
id_max = df.id.max()
date_first = df.loc[df.id==id_min,'time'].values
date_last = df.loc[df.id==id_max,'time'].values
print('There are {} articles in the collection. From id {} on {} to id {} on {}.'.format( 
     df.shape[0],
     id_min, date_first[:10],
     id_max, date_last[:10]))

There are 122720 articles in the collection. From id 4250000.0 on [nan] to id 4372719.0 on ['2018-01-11T20:30:48'].


In [117]:
df.drop(index=df[df.status=='title_not_found'].index).to_pickle('postimees_articles_200917-110118.pkl')

In [118]:
data=pd.read_pickle('postimees_articles_200917-110118.pkl')

In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59305 entries, 1 to 122719
Data columns (total 11 columns):
id          59305 non-null float64
status      59305 non-null object
lang        59305 non-null object
title       59305 non-null object
text        59305 non-null object
theme       59305 non-null object
keywords    59305 non-null object
time        59305 non-null object
comcount    59305 non-null object
autor       59305 non-null object
full_url    59305 non-null object
dtypes: float64(1), object(10)
memory usage: 5.4+ MB


In [120]:
data.id = data.id.astype('int')

In [121]:
data.time = pd.DatetimeIndex(data['time'])

In [122]:
data = data.set_index('id')

In [124]:
data['year'] = pd.DatetimeIndex(data['time']).year
data['month'] = pd.DatetimeIndex(data['time']).month
data['hour'] = pd.DatetimeIndex(data['time']).hour
data['weekday'] = pd.DatetimeIndex(data['time']).weekday

In [125]:
data[data.lang=='et'].theme.value_counts()

Uudised                         8536
Paberleht                       6647
Eesti                           2316
Arvamus                         1427
Järva Teataja                   1317
Sport                           1147
Majandus                        1056
Kirev elu                       1036
Elu24                            883
Maailm                           825
Eesti staarid                    810
Jalgpall                         788
Tasuline                         688
Tartu Postimees                  593
Korvpall                         591
Lemmik                           576
Kodustiil                        469
Äriuudised                       461
Krimi                            448
Kultuur                          434
Suhted & seks                    338
Tarbija                          320
Olümpiauudised                   312
Välismajandus                    300
Ilu & mood                       282
Vaba aeg                         274
PÖFF                             274
G

In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59305 entries, 4250001 to 4372719
Data columns (total 14 columns):
status      59305 non-null object
lang        59305 non-null object
title       59305 non-null object
text        59305 non-null object
theme       59305 non-null object
keywords    59305 non-null object
time        59305 non-null datetime64[ns]
comcount    59305 non-null object
autor       59305 non-null object
full_url    59305 non-null object
year        59305 non-null int64
month       59305 non-null int64
hour        59305 non-null int64
weekday     59305 non-null int64
dtypes: datetime64[ns](1), int64(4), object(9)
memory usage: 6.8+ MB


In [127]:
inx=data[data.text.str.contains('A post shared by')].text.index
inx

Int64Index([4250151, 4250667, 4250671, 4250715, 4250863, 4250945, 4250979,
            4251081, 4251141, 4251207,
            ...
            4371937, 4371985, 4372181, 4372219, 4372225, 4372233, 4372305,
            4372435, 4372477, 4372659],
           dtype='int64', name='id', length=1261)

In [128]:
# В статьях, где есть фраза про источник, переносим эти данные в отдельный столбец
data['from'] = data.text.apply(lambda row: row.split('A post shared by ')[1].split(' on ')[0]
                               if 'A post shared by' in row else "")

In [129]:
# Вырезаем из текста упоминания про источник
start='A post shared by'
end='m P'
data['text'] = data.text.apply(lambda row: "{}{}".format(row.split(start)[0], row.split(end)[-1][3:]) if 'A post shared by' in row else row)

In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59305 entries, 4250001 to 4372719
Data columns (total 15 columns):
status      59305 non-null object
lang        59305 non-null object
title       59305 non-null object
text        59305 non-null object
theme       59305 non-null object
keywords    59305 non-null object
time        59305 non-null datetime64[ns]
comcount    59305 non-null object
autor       59305 non-null object
full_url    59305 non-null object
year        59305 non-null int64
month       59305 non-null int64
hour        59305 non-null int64
weekday     59305 non-null int64
from        59305 non-null object
dtypes: datetime64[ns](1), int64(4), object(10)
memory usage: 7.2+ MB


In [132]:
data

,status,lang,title,text,theme,keywords,time,comcount,autor,full_url,year,month,hour,weekday,from
id,,,,,,,,,,,,,,,
4250001,ok,et,Valgehobusemäel lustis rahvas sportida,Pühapäeval Albu vallas Valgehobusemäe suusa- j...,Paberleht,,2017-09-20 12:41:44,0,0,https://jarvateataja.postimees.ee/4250001/valg...,2017,9,12,2,
4250003,ok,et,Ambla kadus Eesti korvpalliareenilt,Järgmisel nädalal alguse saav korvpalli esilii...,Paberleht,,2017-09-20 12:42:17,0,Kuido Saarpuu,https://jarvateataja.postimees.ee/4250003/ambl...,2017,9,12,2,
4250005,ok,et,Viieaastaseid oodatakse gümnaasiumi,Juba kahel õppeaastal tegutsenud Türi ühisgümn...,Paberleht,,2017-09-20 12:42:34,0,0,https://jarvateataja.postimees.ee/4250005/viie...,2017,9,12,2,
4250007,ok,ru,"Зузу Измаилова: хочу, чтобы мы не обижали друг...",Накануне местных выборов сайт Feministeerium б...,Мнение,"ekre, feministeerium, rail baltica, Варро Воог...",2017-09-21 11:37:29,1,0,https://rus.postimees.ee/4250007/zuzu-izmailov...,2017,9,11,3,
4250009,ok,et,Kiusamisvabade koolide hulka mahub uusi tulijaid,Sihtasutus Kiusamisvaba Kool kutsub koole julg...,Paberleht,,2017-09-20 12:42:57,0,0,https://jarvateataja.postimees.ee/4250009/kius...,2017,9,12,2,
4250011,ok,et,VEPAga lähevad tunnid veel paremaks,Kui Laupa koolis kostab klassist suupillihelis...,Paberleht,,2017-09-20 12:53:24,0,Merit Männi,https://jarvateataja.postimees.ee/4250011/vepa...,2017,9,12,2,
4250013,ok,et,Paide vereloovutuspäevale jagus viis esmadoonorit,Teisipäevast doonoripäeva Paide kultuurikeskus...,Paberleht,,2017-09-20 12:43:48,0,0,https://jarvateataja.postimees.ee/4250013/paid...,2017,9,12,2,
4250015,ok,et,Teadlane: ussitanud seentel pole viga midagi,Mükoloog Leho Tedersoo hinnangul võib tänavuse...,Paberleht,,2017-09-20 12:44:32,0,Jaan Rapp,https://jarvateataja.postimees.ee/4250015/tead...,2017,9,12,2,
4250017,ok,ru,Дмитрий Шепелев в ужасе: «Малахов поступает пр...,У гражданского мужа Жанны Фриске есть весомый ...,TВ,"андрей малахов, дмитрий шепелев",2017-09-20 13:11:09,0,Limon.ee,https://limon.postimees.ee/4250017/dmitriy-she...,2017,9,13,2,


In [133]:
data['url_theme'] = data.full_url.apply(lambda row: row.split('https://')[1].split('.')[0])

In [134]:
data

,status,lang,title,text,theme,keywords,time,comcount,autor,full_url,year,month,hour,weekday,from,url_theme
id,,,,,,,,,,,,,,,,
4250001,ok,et,Valgehobusemäel lustis rahvas sportida,Pühapäeval Albu vallas Valgehobusemäe suusa- j...,Paberleht,,2017-09-20 12:41:44,0,0,https://jarvateataja.postimees.ee/4250001/valg...,2017,9,12,2,,jarvateataja
4250003,ok,et,Ambla kadus Eesti korvpalliareenilt,Järgmisel nädalal alguse saav korvpalli esilii...,Paberleht,,2017-09-20 12:42:17,0,Kuido Saarpuu,https://jarvateataja.postimees.ee/4250003/ambl...,2017,9,12,2,,jarvateataja
4250005,ok,et,Viieaastaseid oodatakse gümnaasiumi,Juba kahel õppeaastal tegutsenud Türi ühisgümn...,Paberleht,,2017-09-20 12:42:34,0,0,https://jarvateataja.postimees.ee/4250005/viie...,2017,9,12,2,,jarvateataja
4250007,ok,ru,"Зузу Измаилова: хочу, чтобы мы не обижали друг...",Накануне местных выборов сайт Feministeerium б...,Мнение,"ekre, feministeerium, rail baltica, Варро Воог...",2017-09-21 11:37:29,1,0,https://rus.postimees.ee/4250007/zuzu-izmailov...,2017,9,11,3,,rus
4250009,ok,et,Kiusamisvabade koolide hulka mahub uusi tulijaid,Sihtasutus Kiusamisvaba Kool kutsub koole julg...,Paberleht,,2017-09-20 12:42:57,0,0,https://jarvateataja.postimees.ee/4250009/kius...,2017,9,12,2,,jarvateataja
4250011,ok,et,VEPAga lähevad tunnid veel paremaks,Kui Laupa koolis kostab klassist suupillihelis...,Paberleht,,2017-09-20 12:53:24,0,Merit Männi,https://jarvateataja.postimees.ee/4250011/vepa...,2017,9,12,2,,jarvateataja
4250013,ok,et,Paide vereloovutuspäevale jagus viis esmadoonorit,Teisipäevast doonoripäeva Paide kultuurikeskus...,Paberleht,,2017-09-20 12:43:48,0,0,https://jarvateataja.postimees.ee/4250013/paid...,2017,9,12,2,,jarvateataja
4250015,ok,et,Teadlane: ussitanud seentel pole viga midagi,Mükoloog Leho Tedersoo hinnangul võib tänavuse...,Paberleht,,2017-09-20 12:44:32,0,Jaan Rapp,https://jarvateataja.postimees.ee/4250015/tead...,2017,9,12,2,,jarvateataja
4250017,ok,ru,Дмитрий Шепелев в ужасе: «Малахов поступает пр...,У гражданского мужа Жанны Фриске есть весомый ...,TВ,"андрей малахов, дмитрий шепелев",2017-09-20 13:11:09,0,Limon.ee,https://limon.postimees.ee/4250017/dmitriy-she...,2017,9,13,2,,limon


In [135]:
data.to_pickle('postimees_59305_articles_200917-110118.pkl')
#data = pd.read_pickle('postimees_59305_articles_200917-110118.pkl')